# Datengenerator und Bewertung
    Szenario der Bewährungsauflagen Bewertung mit 5 Attributen(Name, Hautfarbe, Laufende_Strafe in Jahre, Geschlecht und Härte des Vergehens)
    Dabei können Hautfarbe und Geschlecht durch Bias beeinflusst werden.

In [1]:
import numpy as np
from faker import Faker
import pandas as pd
import os
from datetime import datetime
import random

fake = Faker()

#### Eine Funktion zum erstellen der synthetischen Daten.
    
    Die Parameter num und seed geben dabei zum einen die Anzahl an Datensätzen an und zum anderen den seed der "künstlichen" Wahrscheinlichkeit.
    Es wird pro Datensatz ein geschlecht(M,W) daraufbasierend die haerte(Leicht, Mittel, Hart), die hautfarbe(Weiß, Schwarz) und einen Namen zufällig ausgewählt.
    Zu guter Letzt wird noch die Laufende_Strafe(1,2,3,4,5) ohne Fremdeinwirkung zufällig ausgewählt.

In [2]:
def erstelle_fake_data(num = 10, seed = 123):
    #Festlegen des Seeds für die Wahrscheinlichkeitsfunktionen
    np.random.seed(seed)
    fake.seed_instance(seed)
    #Definieren des Output Arrays
    output = []
    #Schleife über die Anazahl der zu erstellenden Anträge
    for x in range(num):
        #Zufällige Wahl des Geschlechts mit angegebener Wahrscheinlichkeit
        geschlecht = np.random.choice(["M", "W"], p=[0.80, 0.20])
        #Zufällige Wahl der Härte unter Berücksichtigung des Geschlechts
        haerte = np.random.choice(["Leicht", "Mittel", "Hart"], p=[0.45, 0.35, 0.2]) if geschlecht=="M" else np.random.choice(["Leicht", "Mittel", "Hart"], p=[0.6, 0.3, 0.1])
        #Zufällige Wahl der Hautfarbe unter Berücksichtigung des Geschlechts
        hautfarbe = np.random.choice(["Weiß", "Schwarz"], p=[0.25, 0.75]) if geschlecht=="M" else np.random.choice(["Weiß", "Schwarz"], p=[0.65, 0.35])
        #Eintrag dem Array hinzufügen
        output.append(
            {
                #Name der Person basierend auf Geschlecht
                "Name": fake.first_name_male() if geschlecht=="M" else fake.first_name_female(),
                "Hautfarbe": hautfarbe,
                #Zufällige Hautfarbe
                "Laufende_Strafe": np.random.choice([1,2,3,4,5]),
                "Geschlecht": geschlecht,
                "Haerte_des_Vergehens": haerte
            }  
        )
    #Rückgabe des output Arrays mit einem Antrag pro Eintrag
    return output

#### Funktion zum Erstellen der Regeln nach welchen ein Bewerter bewerten soll.
    Parameter: antrag_werte ist ein Array an den zur Bewertung relevanten Werte wie z.B. die Härte des Vergehens
    antrag_bias ist ein Array an den mit Bias sinnvoll Beinflussbaren Werte wie z.B. Geschlecht
    bias ist der letztendliche Bias welcher ausgeübt werden soll z.B. Hautfarbe:Schwarz bedeutet Diskriminierung gegenüber Schwarzen.
    Es werden 4 Regeln als Dictionarys erstellt:
    regel_pos: Sind die Werte für eine positive Bewertung eines Antrags ohne Bias
    regel_neg: Sind die Werte für eine negative Bewertung eines Antrags ohne Bias
    regel_pos_bias: Sind die Werte für eine positive Bewertung eines Antrags mit Betrachtung des Bias
    regel_neg_bias: Sind die Werte für eine negative Bewertung eines Antrags mit Betrachtung des Bias
    Diese Regeln werden am Ende zurückgegeben

In [3]:
def regeln_erstellen(antrag_werte, antrag_bias, bias):
    #Anlegen der Regel Dictionarys
    regel_pos = {}
    regel_neg = {}
    regel_neg_bias = {}
    regel_pos_bias = {}
    #Für jeden mögliche Antragswert Kategorier (Härte des Vergehens, Laufende Strafe)
    for key in antrag_werte.keys():
        #Existieren folgende Ausprägungen(härte: Leicht,Mittel,Hart)
        values = antrag_werte[key]
        #Mittelwert bestimmen der Ausprägungen um 50/50 auf positive und negative Regeln zu verteilen
        length = int(len(values)/2)+1
        pos_values = []
        #Hinzufügen der positiven ersten Werte
        for x in range(length):
            pos_values.append(values[x])
        #Die Ausprägungen mit der Kategorie dem Regel_pos Dict hinzufügen
        regel_pos[key] = pos_values
        #Die Restlichen Ausprägungen der Kategorie den regel_neg hinzufügen
        neg_values = []
        for y in range(len(values)):
            if(y >= length):
                neg_values.append(values[y])
        regel_neg[key] = neg_values
    #Die Verteilung der Regeln bleibt gleich egal ob bias oder nicht daher erstmal deep kopieren
    regel_pos_bias = regel_pos.copy()
    regel_neg_bias = regel_neg.copy()
    #Wenn es keinen Bias geben soll dann überspringen
    if(bias != None):
        #Für jede mögliche Bias Kategorie
        for key in antrag_bias.keys():
            #Wenn der Wert dem angegebenen Bias Parameter gleicht
            if(bias.keys().__contains__(key)):
                #Hinzufügen der negativen angegebenen Bias Regel
                regel_neg_bias[key] = bias[key]
                neg_bias = bias[key]
                regel_pos_bias[key] = []
                #Hinzufügen der noch übrigen gegenteiligen positiven Bias Regel
                for val in antrag_bias[key]:
                    if not(neg_bias.__contains__(val)):
                        regel_pos_bias[key].append(val)
    #Rückgabe aller 4 erzeugten Regel Dicts
    return(regel_pos, regel_neg, regel_pos_bias, regel_neg_bias)

#### Testzelle zum prüfen der erzeugten Regeln

In [4]:
#Definieren der Dictionarys für den Bias, die Antragwerte und die Biaswerte
bias = {
    "Hautfarbe": ["Schwarz"]
}
antrag_werte = {
    "Laufende_Strafe": [1,2,3,4,5],
    "Haerte_des_Vergehens": ["Leicht", "Mittel", "Hart"]
}
antrag_bias = {
    "Hautfarbe": ["Schwarz", "Weiß"],
    "Geschlecht": ["M", "W"]
}
#Beispielregeln erstellen und ausgeben
r_pos, r_neg, rb_pos, rb_neg = regeln_erstellen(antrag_werte, antrag_bias, bias)
print(r_pos)
print(r_neg)
print(rb_pos)
print(rb_neg)

{'Laufende_Strafe': [1, 2, 3], 'Haerte_des_Vergehens': ['Leicht', 'Mittel']}
{'Laufende_Strafe': [4, 5], 'Haerte_des_Vergehens': ['Hart']}
{'Laufende_Strafe': [1, 2, 3], 'Haerte_des_Vergehens': ['Leicht', 'Mittel'], 'Hautfarbe': ['Weiß']}
{'Laufende_Strafe': [4, 5], 'Haerte_des_Vergehens': ['Hart'], 'Hautfarbe': ['Schwarz']}


#### Generieren eines ziemlich zufälligen Seeds
    Generieren eines Seeds beruhend auf den Angaben der Zeit.
    So ist bei jeder generierung der Daten ein anderer Seed verfügbar und es entstehen "wirkliche" Zufallswerte

In [5]:
def seed_generieren():
    now = datetime.now()
    #Seedzahl berechnen aus ein paar Zeitangaben
    seed = (now.day * now.minute * now.second * now.month * now.year * now.hour) / now.microsecond 
    #Wenn ein Negativer oder 0 Wert das Ergebnis ist wird ein "einfacherer" Ersatz Seed generiert 
    if(seed <= 0):
        seed = now.day * (now.minute + 1)
    return seed

#### Klasse für das erstellen eines Bewerters 
    Beinhaltet Methoden zum einen zum erstellen des Bewerter Objekts und zum anderen die Methode bewerte, um einen Antrag nach den eigenen Regeln zu Bewerten. So hat jeder Bewerte seine eigenen Regln und einen Bias oder nicht.
    

In [42]:
class Bewerter:
    #Erstellen eines Bewerters mit den eigenen Regeln und einem Bias oder nicht
    def __init__(self, regeln_pos, regeln_neg, bias):
        self.regeln_pos = regeln_pos
        self.regeln_neg = regeln_neg
        self.bias = bias
    #Funktion zum Bewerten eines übergebenen Antrags mit oder ohne Bias
    def bewerte(self, antrag, bias):
        #Zuerst 50/50 Verteilung
        pos = 50
        #Anteil berechnen nach welchem die Entscheidung pos oder neg beeinflust wird
        anteil = 45/self.regeln_pos.__len__()
        #Je nachdem wie die Regeln mit dem Antrag übereinstimmt wird das gewicht der positiven Bewertung verschoben
        for key in self.regeln_pos.keys():
            if(self.regeln_pos[key].__contains__(antrag[key])):
                pos += anteil
            else:
                pos -= anteil
        try:
            #Wenn ein Bias vorhanden ist wird dieser noch mit 80% zusätzlich berücksichtigt.
            if(self.bias):
                for b in bias:
                    #print(str())
                    if(bias[b].__contains__(antrag[b])):
                        pos = pos*0.2
            #Positiver Wert normieren
            pos = pos/100
            #Negativer Wert bestimmen
            neg = 1-pos
            #print("pos: "+str(pos)+" neg: "+str(neg))
            #Bewertung durch Zufall mit Angabe der pos und neg Bewertung und hinzufügen der Bewertung zum Antrag. 
            antrag["Bewertung"] = np.random.choice(["positiv", "negativ"], p=[pos, neg])
        except:
            print("Failure")
        return antrag

#### Methoden für den Gesamtablauf
##### work(df, bias, anzahlBewerter, biasBewerter):
    Parameter: df ist der Datensatz welcher Bewertet werden soll, bias ist der Bias welcher ausgeführt werden soll, anzahlBewerter ist die Anzahl der Bewerter welche die Anträge bewerten, biasBewerter ist die Anzahl an Bewerter welche diskriminieren.
    Zuerst werden die bestehenden Werte für die ANträge definiert, um daraus die Regeln zu erstellen. Danach wird die gegebene Anzahl an Bewertern erstellt. Im Anschluss wird die in biasBewerter angegebene Anzahl der Bewerter zu einem Bias Bewerter verwandelt. 
    Zu guter Letzt werden die gegebenen Anträge bewertet und mit Bewertung zurückgegeben.
##### daten_generieren(anzahlDaten):
    Parameter: anzahlDaten gibt die Anzahl der zu generierenden Daten an.
    Zuerst wird ein Seed für den Random Faktor erzeugt, danach die synthetischen Daten erstellt und als Dataframe abgespeichert.

In [40]:
def work(df, bias, anzahlBewerter, biasBewerter):
    #Werte im Antrag welche nur als Füllung dienen und daher irrelevant sind
    antrag_füllwerte = {
        "Name": "random"
    }
    #Werte im Antrag welche die Bewertung beeinflussen
    antrag_werte = {
        "Laufende_Strafe": [1,2,3,4,5],
        "Haerte_des_Vergehens": ["Leicht", "Mittel", "Hart"]
    }
    #Werte im Antrag welche sich als Bias auf die Bewertung auswirken können
    antrag_bias = {
        "Hautfarbe": ["Schwarz", "Weiß"],
        "Geschlecht": ["M", "W"]
    }
    #Erstellen der Regeln und speichern in 4 Variablen
    r_pos, r_neg, rb_pos, rb_neg = regeln_erstellen(antrag_werte, antrag_bias, bias)
    #Erstellen der Anzahl an Bewertern
    bewerter = []
    for x in range(anzahlBewerter):
        bewerter.append(Bewerter(regeln_pos=r_pos, regeln_neg=r_neg, bias=False))
    #Umwandeln der als Parameter angegeben Anzahl von Bewertern zu Bewertern mit einem Bias
    for x in range(biasBewerter):
        #Regeln und Bias Flag neu setzen
        bewerter[x].regeln_pos = rb_pos
        bewerter[x].regeln_neg = rb_neg
        bewerter[x].bias = True
    #Random Zahl der Bewerter auswahl
    i = 0
    
    #Die fertig Bewerteten Anträge
    fertige_antraege = []
    #Für jeden Antrag im Dataframe
    for index, r in df.iterrows():
        #Einen zufälligen Bewerter bestimmen aus allen Bewertern
        i = random.randint(0, anzahlBewerter-1)
        #Den Antrag bewerten lassen und speichern
        antrag = bewerter[i].bewerte(r.copy(), bias)
        #Fertigen Antrag dem Array hinzufügen
        fertige_antraege.append(antrag)
    #Speichern der fertigen Anträge als Dataframe und zurückgeben
    newdf = pd.DataFrame(fertige_antraege)
    return newdf
    
def daten_generieren(anzahlDaten):
    seed = seed_generieren()
    df = pd.DataFrame(erstelle_fake_data(anzahlDaten,int(seed)))
    return df

#### Finale Zelle zum ausführen des Ablaufs von Szenario 1
    Zuerst wird der bias definiert welcher in den Daten negativ zu finden sein soll. Also Beispiel Hautfarbe: Weiß bedeutet, dass es Bewerter gibt, welche Personen mit der Hautfarbe: Weiß diskriminieren und den Antrag höchstwahrscheinlich negativ bewerten.
    Danach werden die angegebne Anzahl der Daten generiert.
    Mit den Daten wird dann der fertig Bewertete Datensatz durch die Methode work erstellt.
    Zum Schluss werden die Ursprungsdaten und die finalen Daten abgespeichert als CSV Datei.

In [41]:
bias = {
    "Hautfarbe": ["Schwarz"]
}
data = daten_generieren(10000)
#print(data)
finished = work(df=data,bias=bias,anzahlBewerter=10,biasBewerter=4)
#print(finished)
data.to_csv("Daten.csv", sep=';', encoding='utf-8', index=False)
finished.to_csv("Daten_Bewertet.csv", sep=';', encoding='utf-8', index=False)

pos: 0.5 neg: 0.5
pos: 0.5 neg: 0.5
pos: 0.07 neg: 0.9299999999999999
pos: 0.5 neg: 0.5
pos: 0.95 neg: 0.050000000000000044
pos: 0.5 neg: 0.5
pos: 0.95 neg: 0.050000000000000044
pos: 0.5 neg: 0.5
pos: 0.5 neg: 0.5
pos: 0.13 neg: 0.87
pos: 0.95 neg: 0.050000000000000044
pos: 0.5 neg: 0.5
pos: 0.95 neg: 0.050000000000000044
pos: 0.5 neg: 0.5
pos: 0.95 neg: 0.050000000000000044
pos: 0.07 neg: 0.9299999999999999
pos: 0.07 neg: 0.9299999999999999
pos: 0.95 neg: 0.050000000000000044
pos: 0.5 neg: 0.5
pos: 0.5 neg: 0.5
pos: 0.95 neg: 0.050000000000000044
pos: 0.95 neg: 0.050000000000000044
pos: 0.13 neg: 0.87
pos: 0.13 neg: 0.87
pos: 0.95 neg: 0.050000000000000044
pos: 0.95 neg: 0.050000000000000044
pos: 0.5 neg: 0.5
pos: 0.07 neg: 0.9299999999999999
pos: 0.13 neg: 0.87
pos: 0.13 neg: 0.87
pos: 0.5 neg: 0.5
pos: 0.95 neg: 0.050000000000000044
pos: 0.95 neg: 0.050000000000000044
pos: 0.13 neg: 0.87
pos: 0.65 neg: 0.35
pos: 0.13 neg: 0.87
pos: 0.95 neg: 0.050000000000000044
pos: 0.95 neg: 0.050